# **Title:**
Отчёт по соревнованию: [**cmc2021**](https://www.kaggle.com/c/mmp2021 "Необязательная подсказка")  
Задача на предсказание химических свойств  
30.11.21   
Студент 201 группы ВМК МГУ  
Роман Дьяченко  

## **Abstract:**
Было произведено моделирование ленты, с учётом сдвигов и перемешиваний.  
На этих данных был обучен стэкинг из пяти моделей.

# Исходные Данные:  
  Каждый **объект** обучающий выборки представляет из себя последовательность операций проводимых над рудой на ленте.  
  **Целевая переменная** - химическое свойство руды оставшейся на ленте после проведения операций.
  Типы операций:  
  * Сдвиг ленты на $I$ ячеек. Допустимые значения: $I: {1, 2, 3, 4, 5, 6}$  
  * Добавление руды определённого типа $J$ на одну из ячеек ленты с номером $T$. Допустимые значения:    $T: {0,..., 100}$,   $J: {0, 1, 2, 3}$     
  * Глобальное перемешивание руды на ленте c интенсивностью $V$. Допустимые значения: $V: [0, 3]$   
  * Локальное перемешивание руды с интенсивность $V$ в ячкейке $T$. Допустимые значения: $V: [0, 3]$,    $T: {0,..., 100}$  

Количество операций каждого типа проводимых над лентой не постоянно.  
Суммарное число операций всех типов в одном объекте равно $100$

## **Предобработка данных:**

## Допущения при работе с данными:
* Разные типы руды **располагаются** равномерно в каждой из ячеек
* Перед проведением операций лента пуста

## Глобальное перемешивание камней на ленте:
* Пусть в ячейке с номером $i$ находится $x_{i} ^ {0}$ камней типа $0$.   
* Среднее значение числа камней типа $0$ во всех ячейках равно $E_{0}$.  
Тогда после глобального перемешивания в ячейке с номером $i$  будет $x_{i} + intensive$ $*$ ($E_{0}$ - $x_{i} ^ {0}$).

* Значение $intensive$ определяется как $intensive = ( \frac{V}{3}  - 0.1)^3$, где $V$ - интенсивность из условия задачи, находящаяся в диапозоне $[0, 3]$

Формула для подсчёта $intensive$ была получена при варьировании параметра смещения $0.1$ и степени $3$ с использовании линейной регрессии

## Конечное признаковое описание:
В результате предобработки данных полученое признаковое описание каждого объекта состоящее из 812 признаков.   
Первые $404$ признака - описание количества руды каждого типа в каждой ячейке без учёта глобального перемешивания.
Следующие $404$ признака - описание количества руды каждого типа в каждой ячейке с учётом глобального перемешивания.
Оставшиеся $4$ признака - суммарное количество камней каждого типа на ленте после проведения всех операций

Код трансформирующий исходные запросы в конечную конфигурацию ленты представлен функцией **transform_data**

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import StackingRegressor

In [11]:
df = pd.read_csv('xxx_train.csv')

In [12]:
def transform_data(X):
    conclusion = []
    proc = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]) * X.shape[0]
    for i in range(X.shape[0]):
        if i in proc:
            print(f"Обработано {i} объектов выборки")
        cells = [{0:0, 1:0, 2:0, 3:0} for i in range(101)]
        cells_2 = [{0:0, 1:0, 2:0, 3:0} for i in range(101)]

        for j in range(0, X.shape[1], 3):
            if X.iloc[i, j] == 0:
                step = int(X.iloc[i, j + 1])
                cells[step:] = cells[:-step]
                cells[:step] = [{0:0, 1:0, 2:0, 3:0} for i in range(step)]
                cells_2[step:] = cells_2[:-step]
                cells_2[:step] = [{0:0, 1:0, 2:0, 3:0} for i in range(step)]
            if X.iloc[i, j] == 2:
                intensive =  (X.iloc[i, j + 2] / 3 - 0.1) ** 3
                count_0 = 0
                count_1 = 0
                count_2 = 0
                count_3 = 0
                for cells_k in cells:
                    count_0 += cells_k[0]
                    count_1 += cells_k[1]
                    count_2 += cells_k[2]
                    count_3 += cells_k[3]
                mean_valtype_0 = count_0 / 101
                mean_valtype_1 = count_1 / 101
                mean_valtype_2 = count_2 / 101
                mean_valtype_3 = count_3 / 101
                for cells_k in cells:
                    cells_k[0] += intensive * (mean_valtype_0 - cells_k[0])
                    cells_k[1] += intensive * (mean_valtype_1 - cells_k[1])
                    cells_k[2] += intensive * (mean_valtype_2 - cells_k[2])
                    cells_k[3] += intensive * (mean_valtype_3 - cells_k[3])
            if X.iloc[i, j] == 1:
                cells[int(X.iloc[i, j + 1])][int(X.iloc[i, j + 2])] += 1
                cells_2[int(X.iloc[i, j + 1])][int(X.iloc[i, j + 2])] += 1
        result = []
        result_0 = []
        result_1 = []
        result_2 = []
        result_3 = []
        for j in range(101):
            result_0.append(cells_2[j][0])
            result_1.append(cells_2[j][1])
            result_2.append(cells_2[j][2])
            result_3.append(cells_2[j][3])
            dictar = cells[j]
            result.append(dictar[0])
            result.append(dictar[1])
            result.append(dictar[2])
            result.append(dictar[3])
        sums = [sum(result_0), sum(result_1), sum(result_2), sum(result_3)]
        result = np.array(result_0 +  result_1 + result_2 + result_3 + result + sums)
        conclusion.append(result)
    return pd.DataFrame(conclusion)

In [13]:
X_train = transform_data(df.iloc[:, :-1])
y_train = df.iloc[:, -1]

Обработано 1000 объектов выборки
Обработано 2000 объектов выборки
Обработано 3000 объектов выборки
Обработано 4000 объектов выборки
Обработано 5000 объектов выборки
Обработано 6000 объектов выборки
Обработано 7000 объектов выборки
Обработано 8000 объектов выборки
Обработано 9000 объектов выборки


# **Обучение модели**

In [14]:
estimators = [
    ('lars', linear_model.Lars(n_nonzero_coefs=200, normalize=False)),
    ('ort', linear_model.OrthogonalMatchingPursuit(normalize=False, n_nonzero_coefs = 70)),
    ('ard', linear_model.ARDRegression()),
    ('hub', linear_model.HuberRegressor(epsilon = 1.1)),
    ('neur', MLPRegressor(hidden_layer_sizes = (1624,), activation = 'tanh', solver = 'adam', tol = 0.5,  random_state=1444, max_iter = 12, verbose = True)),
]
reg = StackingRegressor(
    estimators=estimators,
    cv = 5,
    final_estimator= linear_model.LinearRegression(fit_intercept= True)
)

In [ ]:
reg.fit(X_train, y_train)

# **Полученный Score:**
Результат модели на датасете с 100000 объектами (Mean Absolute Error):
* **Public** Leaderboard (5% of the test data): $1.31720$ **(3rd place)**
* **Private** Leaderboard (95% of the test data): $1.29414$ **(4th place)**


# **Вывод**
  Сложно сказать об успешности построенной модели, когда не до конца понимаешь, что ты предсказывал)))  
  Это соревнование научило меня:  
  * работать с основными библиотеками для анализа данных  
  * строить графики (хотя в отчёте не захотелось ничего отражать) 
  * последовательно тестировать гипотезы о законе природы и данных (в отчёт попали самые успешные)   
Ещё я:    
  * узнал много о разных моделях ML и об их коомбинировании  
  * нашёл много интересные статей, курсов и ресурсов по ML, которыми в дальнейшем буду пользоваться
  * вступил в ODS)
